Лабораторная 2. Реализация сверточной нейронной сети для классификации цифр
Исходные данные - база MNIST (http://yann.lecun.com/exdb/mnist/). Обучающая выборка - 70%, тестовая - 30%.
Количество слоев, размерность свертки и гиперпараметры свертки и пулинга на Ваше усмотрение
обучение - обратное распространение ошибки, средняя ошибка - 0,0001
реализовать вычисление матрицы неточностей, по которой должны вычисляться показатели accuracy, precision, recall, F-мера, строится ROC-кривая, вычислятся AUC. Предварительно построить t-SNE и оценить причины ошибочной классификации
DropOut и batchnorm

In [1]:
import numpy as np
import gzip
import os


In [ ]:

def read_idx_file(file_path):
    """ Читает файл в формате IDX (используемый для хранения изображений и меток MNIST).
    Если файла содержит изображения, то он будет интерпретирован как набор изображений, если метки - то как массив меток.

    Параметры:
        file_path (str): Путь к файлу, который нужно прочитать.

    Исключения:
        ValueError: Если формат файла не соответствует ожидаемому.

    Возвращает:
        numpy.ndarray: Массив с изображениями или метками, в зависимости от типа файла.
    """
    with gzip.open(file_path, 'rb') as f:
        # Чтение магического числа
        magic = np.frombuffer(f.read(4), dtype=np.uint32).byteswap()  # Используем byteswap для правильной обработки порядка байтов
        print(f"Read magic number: {magic[0]}")

        if magic == 2051:  # Это изображение
            # Чтение метаданных: количество изображений, строки и столбцы
            num_images = np.frombuffer(f.read(4), dtype=np.uint32).byteswap()[0]
            rows = np.frombuffer(f.read(4), dtype=np.uint32).byteswap()[0]
            cols = np.frombuffer(f.read(4), dtype=np.uint32).byteswap()[0]
            print(f"Images: num_images={num_images}, rows={rows}, cols={cols}")

            # Загружаем все пиксели
            images = np.frombuffer(f.read(), dtype=np.uint8)
            print(f"Loaded {images.size} image pixels.")

            # Проверяем количество пикселей
            expected_pixels = num_images * rows * cols
            if images.size != expected_pixels:
                raise ValueError(f"Mismatch in number of pixels. Expected {expected_pixels}, but got {images.size}.")

            # Преобразуем в массив изображений
            images = images.reshape(num_images, rows, cols)
            return images

        elif magic == 2049:  # Это метки
            num_labels = np.frombuffer(f.read(4), dtype=np.uint32).byteswap()[0]
            labels = np.frombuffer(f.read(), dtype=np.uint8)
            return labels

        else:
            raise ValueError(f"Unknown IDX file format: {magic}")

def load_mnist_data():
    """ Загружает данные MNIST для обучения и тестирования,
    используя функцию 'read_idx_file' для чтения сжатый файлов данных.

    Возвращает:
        tuple:
            - numpy.ndarray: Массив изображений для обучающей выборки, размерность (num_images, rows, cols).
            - numpy.ndarray: Массив меток для обучающей выборки, размерность (num_labels,).
            - numpy.ndarray: Массив изображений для тестовой выборки, размерность (num_images, rows, cols).
            - numpy.ndarray: Массив меток для тестовой выборки, размерность (num_labels,).
    """
    data_dir = './data'  # Указываем путь папки с данными
    # Чтение изображений и меток
    train_images = read_idx_file(os.path.join(data_dir, 'train-images-idx3-ubyte.gz'))
    train_labels = read_idx_file(os.path.join(data_dir, 'train-labels-idx1-ubyte.gz'))
    test_images = read_idx_file(os.path.join(data_dir, 't10k-images-idx3-ubyte.gz'))
    test_labels = read_idx_file(os.path.join(data_dir, 't10k-labels-idx1-ubyte.gz'))

    return train_images, train_labels, test_images, test_labels

# Загрузка данных
X_train, Y_train, X_test, Y_text = load_mnist_data()

# Проверим размерности данных
print(X_train.shape, Y_train.shape, X_test.shape, Y_text.shape)

Read magic number: 2051
Images: num_images=60000, rows=28, cols=28
Loaded 47040000 image pixels.
Read magic number: 2049
Read magic number: 2051
Images: num_images=10000, rows=28, cols=28
Loaded 7840000 image pixels.
Read magic number: 2049
(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [ ]:
# def conv2d(input, filters, biases, stride=1, padding=0):
#     """
#     Применение 2D-свертки.

#     :param input: Входной тензор (batch_size, height, width, channels)
#     :param filters: Фильтры (num_filters, filter_height, filter_width, channels)
#     :param bias: Смещение  для выхода каждого фильтра после свертки (num_filters)
#     :param stride: Шаг свертки
#     :param padding: Отступ для входного изображения
#     :return: Свертка
#     """

#     # Паддинг
#     if padding < 0:
#         raise ValueError("Паддинг должен быть неотрицательным целым числом.")

#     # Размеры входных тензеров
#     batch_size, in_height, in_width, in_channels = input.shape
#     num_filters, filter_height, filter_width, _ = filters.shape

#     # Применяем паддинг, если он задан
#     if padding > 0:
#         input = np.pad(input, ((0,), (padding,), (padding,), (0,)), mode='constant', constant_values=0)

#     # Вычисляем размерности выходного тензора
#     out_height = (in_height - filter_height + 2 * padding) // stride + 1
#     out_width = (in_width - filter_width + 2 * padding) // stride + 1

#     # Используем "strides" для извлечения всех регионов из входного тензора за один шаг
#     # Создаем 3D представление для каждого региона в 2D
#     input_unf = np.lib.stride_tricks.as_strided(input, 
#                                                 shape=(batch_size, out_height, out_width, filter_height, filter_width, in_channels), 
#                                                 strides=(input.strides[0], stride * input.strides[1], stride * input.strides[2], 
#                                                          input.strides[1], input.strides[2], input.strides[3]))

#     # Переводим каждый регион в одномерный вектор
#     input_unf = input_unf.reshape(batch_size, out_height, out_width, -1)

#     # Применяем свертку с фильтрами
#     filters_reshaped = filters.reshape(num_filters, -1)  # Изменяем фильтры в одномерные векторы
#     output = np.tensordot(input_unf, filters_reshaped.T, axes=((3,), (1,)))  # Применяем умножение (векторизация)
    
#     # Добавляем смещения
#     output += biases
    
#     return output


#     # # Выходной тензор
#     # output = np.zeros((batch_size, out_height, out_width, num_filters))

#     # # Применяем фильтры
#     # for i in range(num_filters):
#     #     filter_ = filters[i] # Получаем текущий фильтр
#     #     for j in range(0, in_height - filter_height + 1, stride):
#     #         for k in range(0, in_width - filter_width + 1, stride):
#     #             region = input[:, j:j+filter_height, k:k+filter_width, :]
#     #             output[:, j//stride, k//stride, i] = np.sum(region * filter_, axis=(1, 2, 3))

#     # return output

In [6]:
import matplotlib.pyplot as plt

# Преобразование изображения для свертки (28x28 -> 28x28x1)
def preprocess_image(image):
    # Добавляем ось канала (для одноцветных изображений это будет 1)
    return image.reshape(1, 28, 28, 1)

# Пример простых фильтров (например, для извлечения горизонтальных и вертикальных границ)
def initialize_filters():
    filters = np.zeros((2, 3, 3, 1))  # 2 фильтра размером 3x3 для 1 канала
    filters[0, :, :, 0] = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])  # Фильтр для вертикальных границ
    filters[1, :, :, 0] = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])  # Фильтр для горизонтальных границ
    return filters


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\klimm\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\klimm\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "C:\Users\klimm\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 728, in start
    self.io_lo

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
# Загрузим одно изображение из MNIST для примера
image = X_train[0]  # Берем первое изображение из тренировочных данных

# Преобразуем изображение в нужную форму
image_input = preprocess_image(image)

# Инициализируем фильтры
filters = initialize_filters()

# Инициализируем смещения для каждого фильтра (обычно они небольшие)
biases = np.zeros(2)

# Применим свертку
output = conv2d(image_input, filters, biases, stride=1, padding=0)

# Размерность выходного тензора (batch_size, out_height, out_width, num_filters)
print("Output shape:", output.shape)

# Функция для отображения изображения
def plot_image(image, title="Image"):
    plt.imshow(image.squeeze(), cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Визуализируем исходное изображение
plot_image(image, title="Original Image")

# Визуализируем результаты свертки
# Поскольку у нас два фильтра, вывод будет иметь размер (2, 26, 26)
# Мы можем отобразить каждое из выходных изображений отдельно

for i in range(output.shape[-1]):
    plot_image(output[0, :, :, i], title=f"Filter {i+1} Output")


In [ ]:
def max_pooling(X, pool_size=2, stride=2):
    batch_size, height, width, channels = X.shape
    output_height = (height - pool_size) // stride + 1
    output_width = (width - pool_size) // stride + 1
    
    output = np.zeros((batch_size, output_height, output_width, channels))
    
    for i in range(output_height):
        for j in range(output_width):
            X_slice = X[:, i*stride:i*stride+pool_size, j*stride:j*stride+pool_size, :]
            output[:, i, j, :] = np.max(X_slice, axis=(1, 2))
    
    return output

In [51]:
def flatten(X):
    return X.reshape(X.shape[0], -1)

def dense(input, weights, bias):
    return np.dot(input, weights) + bias

def relu(X):
    return np.maximum(0, X)

def softmax(X):
    exp_X = np.exp(X - np.max(X, axis=1, keepdims=True))
    return exp_X / np.sum(exp_X, axis=1, keepdims=True)

def cross_entropy_loss(y_true, y_pred):
    m = y_true.shape[0]
    return -np.sum(y_true * np.log(y_pred + 1e-9)) / m

In [52]:
def to_categorical(y, num_classes):
    """
    Преобразует метки в формат one-hot encoding.
    :param y: Массив меток (batch_size,)
    :param num_classes: Количество классов
    :return: Массив one-hot encoding (batch_size, num_classes)
    """
    return np.eye(num_classes)[y]


In [53]:
class CNN:
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        
        # Инициализация весов для слоев
        self.conv1_filters = np.random.randn(3, 3, 1, 3) * 0.01  # 3 фильтра 3x3 для 1 канала
        self.conv1_bias = np.zeros(3)
        
        self.conv2_filters = np.random.randn(3, 3, 3, 3) * 0.01  # 3 фильтра 3x3 для 3 каналов
        self.conv2_bias = np.zeros(3)
        
        # Вычисляем размерность после свертки и пулинга
        self.flattened_size = self.calculate_flattened_size(input_shape)
        self.fc_weights = np.random.randn(self.flattened_size, 128) * 0.01  # Размерность после свертки и пулинга
        self.fc_bias = np.zeros(128)
        
        self.output_weights = np.random.randn(128, num_classes) * 0.01
        self.output_bias = np.zeros(num_classes)

    def calculate_flattened_size(self, input_shape):
        # Применяем свертку и пулинг для вычисления выходной размерности
        # В этом примере мы используем stride=1 и padding='valid'
        input = np.zeros((1, *input_shape))  # Подготовим фиктивный тензор для вычислений
        conv1_out = conv2d(input, self.conv1_filters)
        pool1_out = max_pooling(conv1_out)
        conv2_out = conv2d(pool1_out, self.conv2_filters)
        pool2_out = max_pooling(conv2_out)
        return pool2_out.size  # После пулинга мы знаем, какой размер будет у флаттенед тензора

    def forward_pass(self, X):
        # Свертка первого слоя + активация
        self.conv1_out = conv2d(X, self.conv1_filters)
        self.conv1_out = relu(self.conv1_out)
        
        # Слой пулинга
        self.pool1_out = max_pooling(self.conv1_out)
        
        # Свертка второго слоя + активация
        self.conv2_out = conv2d(self.pool1_out, self.conv2_filters)
        self.conv2_out = relu(self.conv2_out)
        
        # Слой пулинга
        self.pool2_out = max_pooling(self.conv2_out)
        
        # Флэттенинг (преобразуем вектор в плоскую форму)
        self.flattened = flatten(self.pool2_out)
        
        # Проверяем размерность после флаттенинга
        print(f"Flattened shape: {self.flattened.shape}")
        
        # Полносвязный слой
        self.fc_out = dense(self.flattened, self.fc_weights, self.fc_bias)
        self.fc_out = relu(self.fc_out)
        
        # Выходной слой
        self.output = dense(self.fc_out, self.output_weights, self.output_bias)
        self.output = softmax(self.output)
        
        return self.output

    def backward_pass(self, X, y, learning_rate):
        m = X.shape[0]
        
        d_output = self.output - y
        d_output /= m
        
        d_output_weights = np.dot(self.fc_out.T, d_output)
        d_output_bias = np.sum(d_output, axis=0)
        
        d_fc_out = np.dot(d_output, self.output_weights.T)
        d_fc_out[self.fc_out <= 0] = 0
        
        d_fc_weights = np.dot(self.flattened.T, d_fc_out)
        d_fc_bias = np.sum(d_fc_out, axis=0)
        
        d_pool2_out = np.dot(d_fc_out, self.fc_weights.T)
        
        d_conv2_out = d_pool2_out.reshape(self.conv2_out.shape)
        d_conv2_out[self.conv2_out <= 0] = 0
        
        d_conv2_filters = np.dot(d_conv2_out, self.pool1_out.T)
        d_conv2_bias = np.sum(d_conv2_out, axis=(0, 1, 2))
        
        d_pool1_out = np.dot(d_conv2_out, self.conv2_filters.T)
        
        d_conv1_out = d_pool1_out.reshape(self.conv1_out.shape)
        d_conv1_out[self.conv1_out <= 0] = 0
        
        d_conv1_filters = np.dot(d_conv1_out, X.T)
        d_conv1_bias = np.sum(d_conv1_out, axis=(0, 1, 2))
        
        # Обновляем веса
        self.conv1_filters -= learning_rate * d_conv1_filters
        self.conv1_bias -= learning_rate * d_conv1_bias
        self.conv2_filters -= learning_rate * d_conv2_filters
        self.conv2_bias -= learning_rate * d_conv2_bias
        self.fc_weights -= learning_rate * d_fc_weights
        self.fc_bias -= learning_rate * d_fc_bias
        self.output_weights -= learning_rate * d_output_weights
        self.output_bias -= learning_rate * d_output_bias

    def train(self, X_train, y_train, learning_rate=0.001, epochs=10, batch_size=32):
        # Убедимся, что X_train имеет правильную форму
        if X_train.shape[-1] != 1:
            X_train = np.expand_dims(X_train, axis=-1)  # Добавляем канал

        num_samples = X_train.shape[0]
        
        # Преобразуем метки в формат one-hot
        y_train = to_categorical(y_train, self.num_classes)
        
        for epoch in range(epochs):
            total_loss = 0
            num_batches = num_samples // batch_size
            for batch in range(num_batches):
                start_idx = batch * batch_size
                end_idx = (batch + 1) * batch_size
                X_batch = X_train[start_idx:end_idx]
                y_batch = y_train[start_idx:end_idx]
                
                predictions = self.forward_pass(X_batch)
                
                loss = cross_entropy_loss(y_batch, predictions)
                total_loss += loss
                
                self.backward_pass(X_batch, y_batch, learning_rate)
            
            avg_loss = total_loss / num_batches
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss}")

In [55]:
import numpy as np

def to_categorical(y, num_classes):
    """
    Преобразует метки в формат one-hot encoding.
    :param y: Массив меток (batch_size,)
    :param num_classes: Количество классов
    :return: Массив one-hot encoding (batch_size, num_classes)
    """
    return np.eye(num_classes)[y]

def cross_entropy_loss(y_true, y_pred):
    m = y_true.shape[0]
    # Используем small epsilon, чтобы избежать логарифма нуля
    return -np.sum(y_true * np.log(y_pred + 1e-9)) / m

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))  # Для стабильности чисел
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

def max_pooling(x, pool_size=2, stride=2):
    batch_size, in_height, in_width, in_channels = x.shape
    out_height = (in_height - pool_size) // stride + 1
    out_width = (in_width - pool_size) // stride + 1
    out = np.zeros((batch_size, out_height, out_width, in_channels))
    for i in range(0, in_height - pool_size + 1, stride):
        for j in range(0, in_width - pool_size + 1, stride):
            region = x[:, i:i+pool_size, j:j+pool_size, :]
            out[:, i//stride, j//stride, :] = np.max(region, axis=(1, 2))
    return out

def conv2d(input, filters, stride=1, padding='valid'):
    batch_size, in_height, in_width, in_channels = input.shape
    num_filters, filter_height, filter_width, _ = filters.shape
    
    # Паддинг
    if padding == 'same':
        pad_height = (in_height - 1) % stride
        pad_width = (in_width - 1) % stride
        input = np.pad(input, ((0, 0), (pad_height//2, pad_height//2), (pad_width//2, pad_width//2), (0, 0)), mode='constant')
    
    out_height = (in_height - filter_height) // stride + 1
    out_width = (in_width - filter_width) // stride + 1
    
    # Выходной тензор
    output = np.zeros((batch_size, out_height, out_width, num_filters))
    
    # Применяем фильтры
    for i in range(num_filters):
        filter_ = filters[i]
        for j in range(0, in_height - filter_height + 1, stride):
            for k in range(0, in_width - filter_width + 1, stride):
                region = input[:, j:j+filter_height, k:k+filter_width, :]
                output[:, j//stride, k//stride, i] = np.sum(region * filter_, axis=(1, 2, 3))
    
    return output

def flatten(x):
    return x.reshape(x.shape[0], -1)

def dense(input, weights, bias):
    return np.dot(input, weights) + bias


class CNN:
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        
        # Инициализация весов для слоев
        self.conv1_filters = np.random.randn(3, 3, 1, 3) * 0.01  # 3 фильтра 3x3 для 1 канала
        self.conv1_bias = np.zeros(3)
        
        self.conv2_filters = np.random.randn(3, 3, 3, 3) * 0.01  # 3 фильтра 3x3 для 3 каналов
        self.conv2_bias = np.zeros(3)
        
        # Рассчитываем размерность после свертки и пулинга
        self.flattened_size = self.calculate_flattened_size(input_shape)
        self.fc_weights = np.random.randn(self.flattened_size, 128) * 0.01  # Размерность после свертки и пулинга
        self.fc_bias = np.zeros(128)
        
        self.output_weights = np.random.randn(128, num_classes) * 0.01
        self.output_bias = np.zeros(num_classes)

    def calculate_flattened_size(self, input_shape):
        # Применяем свертку и пулинг для вычисления выходной размерности
        input = np.zeros((1, *input_shape))  # Подготовим фиктивный тензор для вычислений
        conv1_out = conv2d(input, self.conv1_filters)
        pool1_out = max_pooling(conv1_out)
        conv2_out = conv2d(pool1_out, self.conv2_filters)
        pool2_out = max_pooling(conv2_out)
        return pool2_out.size  # После пулинга мы знаем, какой размер будет у флаттенед тензора

    def forward_pass(self, X):
        # Добавляем канал для входных данных
        if X.shape[-1] != 1:
            X = np.expand_dims(X, axis=-1)
        
        # Свертка первого слоя + активация
        self.conv1_out = conv2d(X, self.conv1_filters)
        self.conv1_out = relu(self.conv1_out)
        
        # Слой пулинга
        self.pool1_out = max_pooling(self.conv1_out)
        
        # Свертка второго слоя + активация
        self.conv2_out = conv2d(self.pool1_out, self.conv2_filters)
        self.conv2_out = relu(self.conv2_out)
        
        # Слой пулинга
        self.pool2_out = max_pooling(self.conv2_out)
        
        # Флэттенинг (преобразуем вектор в плоскую форму)
        self.flattened = flatten(self.pool2_out)
        
        # Полносвязный слой
        self.fc_out = dense(self.flattened, self.fc_weights, self.fc_bias)
        self.fc_out = relu(self.fc_out)
        
        # Выходной слой
        self.output = dense(self.fc_out, self.output_weights, self.output_bias)
        self.output = softmax(self.output)
        
        return self.output

    def backward_pass(self, X, y, learning_rate):
        m = X.shape[0]
        
        d_output = self.output - y
        d_output /= m
        
        d_output_weights = np.dot(self.fc_out.T, d_output)
        d_output_bias = np.sum(d_output, axis=0)
        
        d_fc_out = np.dot(d_output, self.output_weights.T)
        d_fc_out[self.fc_out <= 0] = 0
        
        d_fc_weights = np.dot(self.flattened.T, d_fc_out)
        d_fc_bias = np.sum(d_fc_out, axis=0)
        
        d_pool2_out = np.dot(d_fc_out, self.fc_weights.T)
        
        d_conv2_out = d_pool2_out.reshape(self.conv2_out.shape)
        d_conv2_out[self.conv2_out <= 0] = 0
        
        d_conv2_filters = np.dot(d_conv2_out, self.pool1_out.T)
        d_conv2_bias = np.sum(d_conv2_out, axis=(0, 1, 2))
        
        d_pool1_out = np.dot(d_conv2_out, self.conv2_filters.T)
        
        d_conv1_out = d_pool1_out.reshape(self.conv1_out.shape)
        d_conv1_out[self.conv1_out <= 0] = 0
        
        d_conv1_filters = np.dot(d_conv1_out, X.T)
        d_conv1_bias = np.sum(d_conv1_out, axis=(0, 1, 2))
        
        # Обновляем веса
        self.conv1_filters -= learning_rate * d_conv1_filters
        self.conv1_bias -= learning_rate * d_conv1_bias
        self.conv2_filters -= learning_rate * d_conv2_filters
        self.conv2_bias -= learning_rate * d_conv2_bias
        self.fc_weights -= learning_rate * d_fc_weights
        self.fc_bias -= learning_rate * d_fc_bias
        self.output_weights -= learning_rate * d_output_weights
        self.output_bias -= learning_rate * d_output_bias

    def train(self, X_train, y_train, learning_rate=0.001, epochs=10, batch_size=32):
        # Убедимся, что X_train имеет правильную форму
        if X_train.shape[-1] != 1:
            X_train = np.expand_dims(X_train, axis=-1)  # Добавляем канал

        num_samples = X_train.shape[0]
        
        # Преобразуем метки в формат one-hot
        y_train = to_categorical(y_train, self.num_classes)
        
        for epoch in range(epochs):
            total_loss = 0
            num_batches = num_samples // batch_size
            for batch in range(num_batches):
                start_idx = batch * batch_size
                end_idx = (batch + 1) * batch_size
                X_batch = X_train[start_idx:end_idx]
                y_batch = y_train[start_idx:end_idx]
                
                predictions = self.forward_pass(X_batch)
                
                loss = cross_entropy_loss(y_batch, predictions)
                total_loss += loss
                
                self.backward_pass(X_batch, y_batch, learning_rate)
            
            avg_loss = total_loss / num_batches
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss}")


In [ ]:
# Нормализация изображений в диапазон [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Преобразование изображений в форму (batch_size, height, width, channels)
X_train = np.expand_dims(X_train, axis=-1)  # (60000, 28, 28, 1)
X_test = np.expand_dims(X_test, axis=-1)    # (10000, 28, 28, 1)

# Преобразование меток в формат one-hot
Y_train = to_categorical(Y_train, 10)  # 10 классов
Y_test = to_categorical(Y_test, 10)



In [56]:
# Инициализация модели
model = CNN(input_shape=(28, 28, 1), num_classes=10)

# Преобразование данных в 4D тензор
X_train = np.expand_dims(X_train, axis=-1)  # Добавляем канал, если это одноцветные изображения
X_test = np.expand_dims(X_test, axis=-1)

# Обучение модели
model.train(X_train, Y_train, learning_rate=0.001, epochs=10, batch_size=32)



ValueError: too many values to unpack (expected 4)